In [192]:
import pandas as pd

In [193]:
group_W_1 = ['NC','40', '30/5', '30/4', '30/3', '30/2', '30/1']
group_W_2 = ['30', '15/5', '15/4', '15/3', '15/2','15/1']

group_M_1 = ['NC', '40', '30/5', '30/4', '30/3', '30/2', '30/1']
group_M_2 = ['30', '15/5', '15/4']
group_M_3 = ['15/3', '15/2','15/1']
group_M_4 = ['15','5/6','4/6','3/6','2/6','1/6','0']

list_classement = ['NC', '40', '30/5', '30/4', '30/3', '30/2', '30/1','30', '15/5',
                   '15/4','15/3', '15/2','15/1',
        '15','5/6','4/6','3/6','2/6','1/6','0']

list_tableau = ['tableau final sm','tableau final sm+35','tableau final sm+45',
                'fin de tableau sm 3ème série', 'tableau final sd', 'fin de tableau sd 3ème série',
               'fin de tableau 4ème série']

In [194]:
categories = ['simple messieurs senior', 'simple messieurs 35', 'simple messieurs 45',
              'simple dames senior', 'simple messieurs 55']

In [195]:
def extract_cat(file, categories):
    extracted = dict()
    cat_present = []

    for cat in categories:
        if cat in file['Epreuve'].tolist():
            extracted[cat] = file.loc[file['Epreuve'] == cat,:]
            cat_present.append(cat)
    return extracted,cat_present

In [225]:
def categ(cat, classement):
    if 'messieurs' in cat and '45' not in cat:
        if classement in group_M_1:
            val = 'NC_30_1'
        elif classement in group_M_2:
            val = '30_15_4' 
        elif classement in group_M_3:
            val = '15_3_15_1'
        else:
            val = 'eject'
    else:
        if classement in group_W_1:
            val = 'NC_30_1'
        elif classement in group_W_2:
            val = '30_15_1'
        else:
            val = 'eject'
    return val

In [245]:
def serie(val_serie, VD, tableau, extract,i):
    val = 0
    
    comp = extract['Tour/Poule'].tolist()
    print(tableau)
    print(comp)
    if '1/4 finale' in comp and '1/2 finale' in comp and 'finale' in comp:
        if val_serie.lower() == '1/4 finale' and tableau in list_tableau:
            val = 1
            
        elif val_serie.lower() == '1/2 finale'and tableau in list_tableau:
            val = 2

        elif val_serie.lower() == 'finale' and tableau in list_tableau and VD == 0:
            val = 2

        elif val_serie.lower() == 'finale' and VD == 1 and tableau in list_tableau:
            val = 5

        else:
            val = 0
    
    elif '1/2 finale' in comp and 'finale' in comp:
        if val_serie.lower() == '1/2 finale'and tableau in list_tableau:
            val = 3

        elif val_serie.lower() == 'finale' and tableau in list_tableau and VD == 0:
            val = 2

        elif val_serie.lower() == 'finale' and VD == 1 and tableau in list_tableau:
            val = 5

        else:
            val = 0
    
    
    elif 'finale' in comp:
        print(extract['Nom 1'][i])
        print(extract['Nom 2'][i])
        if val_serie.lower() == 'finale' and tableau in list_tableau and VD == 0:
            val = 5

        elif val_serie.lower() == 'finale' and VD == 1 and tableau in list_tableau:
            val = 8

        else:
            val = 0
    
    return val

In [246]:
def assign_points(extracted,cat_present):
    for cat in cat_present:
        name_list = []
        score_list = []
        cat_list = []
        tour_score = []
        
        for i in extracted[cat].index:
            nom_1 = extracted[cat].loc[i,'Nom 1']
            nom_2 = extracted[cat].loc[i,'Nom 2']
            VD = extracted[cat].loc[i,'Résultat']
            Classt_1 = extracted[cat].loc[i,'Classt 1']
            Classt_1 = Classt_1.translate({ord(' '): None})
            
            Classt_2 = extracted[cat].loc[i,'Classt 2']
            Classt_2 = Classt_2.translate({ord(' '): None})
            Score = extracted[cat].loc[i,'Score']
            Phase = extracted[cat].loc[i,'Phase']
            Tour = extracted[cat].loc[i,'Tour/Poule']
            
            ind_1 = list_classement.index(Classt_1)
            ind_2 = list_classement.index(Classt_2)
            
            count_name_1 = extracted[cat]['Nom 1'].str.count(nom_1).sum()
            count_name_1 = count_name_1 + extracted[cat]['Nom 2'].str.count(nom_1).sum()
            
            count_name_2 = extracted[cat]['Nom 2'].str.count(nom_2).sum()
            count_name_2 = count_name_2 + extracted[cat]['Nom 1'].str.count(nom_2).sum()
            
            if VD == 'v' and Score == 'wo':
                name_list.append(nom_1)
                score_list.append(1)
                cat_list.append(categ(cat,Classt_1))
                tour_score.append(serie(Tour,1,Phase,extracted[cat],i))
               
                if count_name_2>1:
                    
                    name_list.append(nom_2)
                    score_list.append(0)
                    cat_list.append(categ(cat,Classt_2))
                    tour_score.append(serie(Tour,0,Phase,extracted[cat],i))
                
            elif VD == 'd' and Score =='wo':
                name_list.append(nom_2)
                score_list.append(1)
                cat_list.append(categ(cat,Classt_2))
                tour_score.append(serie(Tour,1,Phase,extracted[cat],i))
                
                if count_name_1 > 1:
                    name_list.append(nom_1)
                    score_list.append(0)
                    cat_list.append(categ(cat,Classt_1))
                    tour_score.append(serie(Tour,0,Phase,extracted[cat],i))
                     
            elif Score != 'wo':
                if VD == 'v':
                    if ind_1 - ind_2 <= -1:
                        name_list.append(nom_1)
                        score_list.append(3)
                        cat_list.append(categ(cat,Classt_1))
                        tour_score.append(serie(Tour,1,Phase,extracted[cat],i))
                        
                    elif ind_1 - ind_2 == 0:
                        name_list.append(nom_1)
                        score_list.append(2)
                        cat_list.append(categ(cat,Classt_1))
                        tour_score.append(serie(Tour,1,Phase,extracted[cat],i))
                    
                    elif ind_1 - ind_2 > 0:
                        name_list.append(nom_1)
                        score_list.append(1)
                        cat_list.append(categ(cat,Classt_1))
                        tour_score.append(serie(Tour,1,Phase,extracted[cat],i))
                    
                    name_list.append(nom_2)
                    score_list.append(0)
                    cat_list.append(categ(cat,Classt_2))
                    tour_score.append(serie(Tour,0,Phase,extracted[cat],i))
                        
                elif VD == 'd':
                    if ind_2 - ind_1 <= -1:
                        name_list.append(nom_2)
                        score_list.append(3)
                        cat_list.append(categ(cat,Classt_2))
                        tour_score.append(serie(Tour,1,Phase,extracted[cat],i))
                        
                    elif ind_2 - ind_1 == 0:
                        name_list.append(nom_2)
                        score_list.append(2)
                        cat_list.append(categ(cat,Classt_2))
                        tour_score.append(serie(Tour,1,Phase,extracted[cat],i))
                    
                    elif ind_2 - ind_1 > 0:
                        name_list.append(nom_2)
                        score_list.append(1)
                        cat_list.append(categ(cat,Classt_2))
                        tour_score.append(serie(Tour,1,Phase,extracted[cat],i))
                    
                    name_list.append(nom_1)
                    score_list.append(0)
                    cat_list.append(categ(cat,Classt_1))
                    tour_score.append(serie(Tour,0,Phase,extracted[cat],i))
                    
        res = pd.DataFrame(score_list, index = name_list, columns = ['Score'])
        res['Rank'] = cat_list
        res['Tour'] = tour_score
        res['Score'] = res['Score'] + res['Tour']
        extracted[cat+' Score'] = res  
        #print(extracted[cat+' Score'])
    return(extracted)

In [247]:
def get_ranking(extracted, cat_present):
    for cat in cat_present:
        extracted[cat+' Score'] = extracted[cat+' Score'][extracted[cat+' Score'].Rank != 'eject']
        
        rank_list = []
        for rank in extracted[cat+' Score'].Rank:
            if rank not in rank_list:
                rank_list.append(rank)
        
        for rank in rank_list:
            extract_rank = extracted[cat+' Score'][extracted[cat+' Score'].Rank == rank]
            extracted[cat+' Classement '+ rank] = extract_rank.groupby(extract_rank.index).agg({'Score':sum})
            extracted[cat+' Classement '+ rank] = extracted[cat+' Classement '+ rank].sort_values(by=['Score'], ascending = False)
            #print(cat+' Classement '+ rank)
            #print(extracted[cat+' Classement '+ rank])
    return(extracted)

In [248]:
#extracted,cat_present = extract_cat(file, categories)
#extracted = assign_points(extracted,cat_present)
#extracted = get_ranking(extracted, cat_present)

demander fichier courcelle
calculer res pour les differents et appliquer point bonus participation tournois

In [249]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('Data_2020/') if isfile(join('Data_2020/', f))]
onlyfiles

['Courcelles_2020.csv', '.DS_Store', 'Copie-de-Matches.xls', 'Avion_2020.pdf']

In [250]:
data = dict()
extracted = dict()
list_places = []

for file in onlyfiles:
    if file.endswith(".csv"):
        sp_file = file.split('.')
        data[sp_file[0]] = pd.read_csv('Data_2020/'+file, sep =";")
        
        data[sp_file[0]]['Epreuve'] = data[sp_file[0]]['Epreuve'].str.lower()
        data[sp_file[0]]['Phase'] = data[sp_file[0]]['Phase'].str.lower()
        data[sp_file[0]]['Tour/Poule'] = data[sp_file[0]]['Tour/Poule'].str.lower()
        data[sp_file[0]]['Nom 1'] = data[sp_file[0]]['Nom 1'].str.lower()
        data[sp_file[0]]['Nom 2'] = data[sp_file[0]]['Nom 2'].str.lower()
        data[sp_file[0]]['Résultat'] = data[sp_file[0]]['Résultat'].str.lower()
        
        list_places.append(sp_file[0])
        
        #print(sp_file)
        extracted[sp_file[0]],cat_present = extract_cat(data[sp_file[0]], categories)
        extracted[sp_file[0]] = assign_points(extracted[sp_file[0]],cat_present)
        extracted[sp_file[0]] = get_ranking(extracted[sp_file[0]], cat_present)

nc à 30/1
['tour 1', 'tour 1', 'tour 1', 'tour 2', 'tour 2', 'tour 2', 'tour 2', 'tour 3', 'tour 3', 'tour 3', 'tour 1', 'tour 2', 'tour 2', 'tour 2', 'tour 3', 'tour 3', 'tour 3', 'tour 1', 'tour 2', 'tour 1', 'tour 2', 'tour 3', 'tour 1', '1/8 finale', 'tour 2', 'tour 2', '1/8 finale', '1/8 finale', 'tour 2', '1/8 finale', '1/4 finale', '1/4 finale', '1/8 finale', '1/4 finale', '1/4 finale', '1/2 finale', '1/2 finale', '1/2 finale', 'finale', 'finale']
nc à 30/1
['tour 1', 'tour 1', 'tour 1', 'tour 2', 'tour 2', 'tour 2', 'tour 2', 'tour 3', 'tour 3', 'tour 3', 'tour 1', 'tour 2', 'tour 2', 'tour 2', 'tour 3', 'tour 3', 'tour 3', 'tour 1', 'tour 2', 'tour 1', 'tour 2', 'tour 3', 'tour 1', '1/8 finale', 'tour 2', 'tour 2', '1/8 finale', '1/8 finale', 'tour 2', '1/8 finale', '1/4 finale', '1/4 finale', '1/8 finale', '1/4 finale', '1/4 finale', '1/2 finale', '1/2 finale', '1/2 finale', 'finale', 'finale']
nc à 30/1
['tour 1', 'tour 1', 'tour 1', 'tour 2', 'tour 2', 'tour 2', 'tour 2', '

30 à 15/4
['tour 2', 'tour 1', 'tour 1', 'tour 1', 'tour 2', 'tour 2', 'tour 2', 'tour 3', 'tour 3', 'tour 3', 'tour 3', '1/4 finale', '1/4 finale', '1/4 finale', '1/4 finale', '1/2 finale', '1/2 finale', 'finale']
30 à 15/4
['tour 2', 'tour 1', 'tour 1', 'tour 1', 'tour 2', 'tour 2', 'tour 2', 'tour 3', 'tour 3', 'tour 3', 'tour 3', '1/4 finale', '1/4 finale', '1/4 finale', '1/4 finale', '1/2 finale', '1/2 finale', 'finale']
30 à 15/4
['tour 2', 'tour 1', 'tour 1', 'tour 1', 'tour 2', 'tour 2', 'tour 2', 'tour 3', 'tour 3', 'tour 3', 'tour 3', '1/4 finale', '1/4 finale', '1/4 finale', '1/4 finale', '1/2 finale', '1/2 finale', 'finale']
30 à 15/4
['tour 2', 'tour 1', 'tour 1', 'tour 1', 'tour 2', 'tour 2', 'tour 2', 'tour 3', 'tour 3', 'tour 3', 'tour 3', '1/4 finale', '1/4 finale', '1/4 finale', '1/4 finale', '1/2 finale', '1/2 finale', 'finale']
tableau final sm+45
['tour 2', 'tour 1', 'tour 1', 'tour 1', 'tour 2', 'tour 2', 'tour 2', 'tour 3', 'tour 3', 'tour 3', 'tour 3', '1/4 fina

In [251]:
list_name = dict()
classt_tot = dict()

for file in onlyfiles:
    if file.endswith(".csv"):
        sp_file = file.split('.')
        data[sp_file[0]] = pd.read_csv('Data_2020/'+file, sep =";")
        
        list_name[sp_file[0]] = []
        classt_tot[sp_file[0]] = []
        print(sp_file[0])
        
        for i in range(data[sp_file[0]].shape[0]):
            nom_1 = data[sp_file[0]].loc[i, 'Nom 1']
            nom_2 = data[sp_file[0]].loc[i, 'Nom 2']
            res = data[sp_file[0]].loc[i, 'Score']
            VD = data[sp_file[0]].loc[i, 'Résultat']
            classt_1 = data[sp_file[0]].loc[i, 'Classt 1']
            classt_2 = data[sp_file[0]].loc[i, 'Classt 2']
            
            if nom_1 not in list_name[sp_file[0]] and '/' not in nom_1 and res != 'wo':
                list_name[sp_file[0]].append(nom_1)
                classt_tot[sp_file[0]].append(classt_1)
                
            if nom_2 not in list_name[sp_file[0]] and '/' not in nom_2 and res != 'wo':
                list_name[sp_file[0]].append(nom_2)
                classt_tot[sp_file[0]].append(classt_2)
            
            if res == 'wo':
                if VD == 'D' and nom_2 not in list_name[sp_file[0]] and '/' not in nom_2:
                    list_name[sp_file[0]].append(nom_2)
                    classt_tot[sp_file[0]].append(classt_2)
                    
                if VD == 'V' and nom_1 not in list_name[sp_file[0]] and '/' not in nom_1:
                    list_name[sp_file[0]].append(nom_1)
                    classt_tot[sp_file[0]].append(classt_1)

Courcelles_2020


In [252]:
total_names = []
total_classt =[]
for loc in list_places:
    total_names = total_names + list_name[loc]
    total_classt = total_classt + classt_tot[loc]

In [253]:
presence_point = pd.DataFrame(index = total_names)

In [254]:
point = []
lower_name = []
nb_match = []

for name in total_names:
    lower_name.append(name.lower())
    
recap_point = pd.DataFrame(index = lower_name)

for name in total_names:
    appear = total_names.count(name)
    if appear == 1:
        point.append(1)
        nb_match.append(appear)
    if appear == 2:
        point.append(2)
        nb_match.append(appear)
    if appear == 3:
        point.append(4)
        nb_match.append(appear)
    if appear == 4:
        point.append(6)
        nb_match.append(appear)
    if appear == 5:
        point.append(9)
        nb_match.append(appear)
    if appear == 6:
        point.append(12)
        nb_match.append(appear)
    if appear == 7:
        point.append(16)
        nb_match.append(appear)
    if appear == 8:
        point.append(20)
        nb_match.append(appear)
    if appear >= 9:
        point.append(20 + 5*(appear - 8))
        nb_match.append(appear)
recap_point['Point_nb_match'] = point
recap_point['Classement'] = total_classt
recap_point['Nombre matchs'] = nb_match

In [255]:
recap_point

,Point_nb_match,Classement,Nombre matchs
ornet lydie,1,30/2,1
bataille patricia,1,30/1,1
bellahcene samir,1,30/3,1
terzoutyettou nahel,1,30/3,1
bernard tony,1,30/4,1
vanhoutte david,1,30/5,1
krol kevin,1,30/5,1
taillez julien,1,30/5,1
casamajor alexandre,1,30/2,1
mathorel cyril,1,30/3,1


In [237]:
recap_point = recap_point[~recap_point.index.duplicated(keep='first')]

In [209]:
list_tab_final = ['simple messieurs senior Classement NC_30_1', 'simple messieurs senior Classement 30_15_4',
                 'simple messieurs senior Classement 15_3_15_1','simple messieurs 35 Classement NC_30_1',
                 'simple messieurs 35 Classement 30_15_4','simple messieurs 35 Classement 15_3_15_1',
                 'simple messieurs 45 Classement NC_30_1','simple messieurs 45 Classement 30_15_1',
                 'simple dames senior Classement NC_30_1', 'simple dames senior Classement 30_15_1']

In [210]:
recap_tab = dict()

for cat in list_tab_final:
    list_name = []
    for loc in list_places:
        for name in extracted[loc][cat].index:
            if name not in list_name:
                list_name.append(name)
                
    recap_tab[cat] = pd.DataFrame(index = list_name)
    
    for loc in list_places:    
        recap_tab[cat]['Score ' + loc] = extracted[loc][cat]

    recap_tab[cat]['Classement'] = recap_point['Classement']
    recap_tab[cat]['Points participation'] = recap_point['Point_nb_match']
    recap_tab[cat]['Score total'] = recap_tab[cat].sum(axis =1)
    recap_tab[cat]['Nombre matchs'] = recap_point['Nombre matchs']
    recap_tab[cat] = recap_tab[cat].fillna(0)
    
    recap_tab[cat] = recap_tab[cat].sort_values(by = ['Score total'], ascending = False)

    final_tab = pd.DataFrame()

    final_tab['Rang'] = range(1,recap_tab[cat].shape[0] + 1)

    final_tab['Nom/Prenom'] = recap_tab[cat].index
    
    title_name = []
    for name in final_tab['Nom/Prenom']:
        title_name.append(name.title())
        
    final_tab['Nom/Prenom'] = title_name
    final_tab['Classement'] = recap_tab[cat]['Classement'].values
    final_tab['Nb de tournois joués'] = recap_tab[cat]['Nombre matchs'].values
    final_tab['Points'] = recap_tab[cat]['Score total'].values

    final_tab.to_excel('Resultat/'+cat+'.xlsx')


In [211]:
recap_tab['simple dames senior Classement 30_15_1']

,Score Courcelles_2020,Classement,Points participation,Score total,Nombre matchs
bataille julie,7,15/3,1,8,1
debailleul marion,6,15/1,1,7,1
hombert marine,2,15/4,1,3,1
pistone loulia,1,15/5,1,2,1
petit séverine,0,30,1,1,1


In [212]:
final_tab

,Rang,Nom/Prenom,Classement,Nb de tournois joués,Points
0,1,Bataille Julie,15/3,1,8
1,2,Debailleul Marion,15/1,1,7
2,3,Hombert Marine,15/4,1,3
3,4,Pistone Loulia,15/5,1,2
4,5,Petit Séverine,30,1,1
